In [2]:
import optuna
import torch
import numpy as np
import tqdm
import sklearn
import networkx as nx
import random
import warnings

/home/younes/miniconda3/envs/ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Model definition

In [2]:
class AutoEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.encoder = torch.nn.Linear(input_dim, hidden_dim)
        self.decoder = torch.nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        encoded = torch.sigmoid(self.encoder(x))
        decoded = torch.sigmoid(self.decoder(encoded))
        return encoded, decoded

In [3]:
class GraphEncoder(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims):
        super().__init__()
        self.autoencoders = torch.nn.ModuleList()
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            self.autoencoders.append(AutoEncoder(prev_dim, hidden_dim))
            prev_dim = hidden_dim

    def forward(self, x):
        for autoencoder in self.autoencoders:
            x = torch.sigmoid(autoencoder.encoder(x))
        encoded = x
        for autoencoder in reversed(self.autoencoders):
            x = torch.sigmoid(autoencoder.decoder(x))
        decoded = x
        return encoded, decoded

# 2. Test on benchmark "Wine"

In [4]:
def compute_ncut(s, labels):
    """
    Compute  normalized cut for given similarity matrix s and cluster labels:
      Ncut = sum_k cut(C_k, V\C_k) / assoc(C_k, V)
    where
      cut(C, V\C) = sum_{i in C, j not in C} A[i,j]
      assoc(C, V) = sum_{i in C, j in V} A[i,j]  (i.e., volume of C)
    A : symmetric adjacency/similarity numpy array
    labels : length-n array of integer cluster labels
    Returns float Ncut value.
    """

    # Get the unique labels in the community assignment
    unique_labels = np.unique(labels)
    
    # Precompute degrees
    degrees = s.sum(axis=1)  # degree/volume per node
    
    # Initialize ncut
    ncut = 0.0
    
    # For each cluster compute link and volume, then sum up to get ncut
    for lab in unique_labels:
        
        # Get the indices of nodes in cluster lab
        idx = np.where(labels == lab)[0]
        if idx.size == 0:
            raise Exception("compute_ncut_from_labels: empty cluster found in labels.")
        
        # Compute volume = sum of degrees of nodes in idx
        volume = degrees[idx].sum()
        
        # If volume is not zero, compute link to get the local cut then sum to ncut, otherwise skip (i.e. cut = 0)
        if volume != 0:

            # Compute link = sum over i in C, j not in C, of A[i,j]
            # = volume - internal connections
            internal_connections = s[np.ix_(idx, idx)].sum()
            link = volume - internal_connections
            
            # Compute local cut contribution
            local_cut = link / volume

            # Sum to ncut
            ncut += local_cut
    
    return ncut

warnings.filterwarnings("error", category=sklearn.exceptions.ConvergenceWarning)

## 2.1. Data loading

In [5]:
# Loading Wine
x, y= sklearn.datasets.load_wine(return_X_y=True, as_frame=False)
x = sklearn.preprocessing.MinMaxScaler().fit_transform(x)
s = sklearn.metrics.pairwise.cosine_similarity(x, x)
nts = s / np.sum(s, axis=1, keepdims=True)
print("[*] nts.shape:", nts.shape)
print("[*] number of clusters:", len(set(y)))
cumulated_nmi = 0
cumulated_ncut = 0
nb_kmeans_tests = 100
random.seed(0)
for _ in tqdm.tqdm(range(nb_kmeans_tests)):
    kmeans = sklearn.cluster.KMeans(n_clusters=len(set(y)), algorithm="lloyd", random_state=random.randint(0, 10000))
    y_pred_origspace = kmeans.fit_predict(nts)
    cumulated_nmi += sklearn.metrics.normalized_mutual_info_score(y, y_pred_origspace)
    cumulated_ncut += compute_ncut(s, y_pred_origspace)
print("[*] original space average nmi:", cumulated_nmi / nb_kmeans_tests)
print("[*] original space average ncut:", cumulated_ncut / nb_kmeans_tests)

[*] nts.shape: (178, 178)
[*] number of clusters: 3


100%|██████████| 100/100 [00:00<00:00, 284.20it/s]

[*] original space average nmi: 0.6351524906645806
[*] original space average ncut: 1.898660189065159


In [6]:
cumulated_nmi = 0
cumulated_ncut = 0
nb_kmeans_tests = 100
random.seed(0)
for _ in tqdm.tqdm(range(nb_kmeans_tests)):
    y_pred_origspace = y_pred_origspace = sklearn.cluster.SpectralClustering(n_clusters=len(set(y)), affinity='precomputed', assign_labels='kmeans', random_state=random.randint(0, 10000)).fit_predict(s)
    cumulated_nmi += sklearn.metrics.normalized_mutual_info_score(y, y_pred_origspace)
    cumulated_ncut += compute_ncut(s, y_pred_origspace)
print("[*] original space average nmi:", cumulated_nmi / nb_kmeans_tests)
print("[*] original space average ncut:", cumulated_ncut / nb_kmeans_tests)

100%|██████████| 100/100 [00:01<00:00, 98.20it/s]

[*] original space average nmi: 0.7129036297699397
[*] original space average ncut: 1.8964924406149422


In [8]:
def objective(trial):

    # Print trial number
    print(f"\ntrial {trial.number}----------------------------")
    
    # Set globals
    global best_avg_nmi
    global best_avg_ncut
    global best_avg_ncut_avg_nmi
    
    # Set random seeds
    torch.manual_seed(97)
    np.random.seed(97)
    random.seed(97)
    
    # Create the model using the hidden dimensions
    hidden_dims = [128, 64]
    model = GraphEncoder(input_dim=x_train.shape[1], hidden_dims=[128,64]).to(device)

    # Suggest rho and beta for the sparsity constraint
    rho = trial.suggest_float("rho", 1e-4, 1e-1, log=True)
    beta = trial.suggest_float("beta", 1e-2, 1e3, log=True)
    
    # Suggest a learning rate for the optimizer and create the optimizer    
    lr = trial.suggest_float("lr", 1e-3, 1e-2, log=True)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    
    # Create initial dataloader
    current_x_train = x_train.clone().to(device)
    dataloader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(current_x_train),
        batch_size=batch_size,
        shuffle=True
    )
    dataloader_iter = iter(dataloader)
    # Suggest nb_epochs_per_layer
    nb_epochs_per_layer = nb_epochs_per_layer_pool[trial.suggest_int("nb_epochs_per_layer", 0, len(nb_epochs_per_layer_pool)-1)]
    nb_train_iters = nb_epochs_per_layer * len(dataloader)

    # Print some hyper parameters
    print("> hidden dims =", hidden_dims)
    print("> rho =", rho)
    print("> beta =", beta)
    
    # Launch the training loop
    # For each layer in the stacked autoencoder: train the layer
    for layer_number in range(len(model.autoencoders)):
        for _ in (pb := tqdm.tqdm(range(nb_train_iters), desc=f"layer: {layer_number}")):
            try:
                (x_batch,) = next(dataloader_iter)
            except StopIteration:
                dataloader_iter = iter(dataloader)
                (x_batch,) = next(dataloader_iter)
            x_batch = x_batch.to(device)
            optimizer.zero_grad()
            encoded, decoded = model.autoencoders[layer_number](x_batch)
            loss_1 = torch.nn.functional.mse_loss(decoded, x_batch, reduction='sum')
            rho_hat = torch.mean(encoded, dim=0)
            loss_2 = torch.sum(rho * torch.log(rho / rho_hat) + (1 - rho) * torch.log((1 - rho) / (1 - rho_hat)))
            loss = loss_1 + beta * loss_2
            loss.backward()
            optimizer.step()
            pb.set_postfix({"loss": loss.item()})

        # Create new dataloader on the latent representations
        with torch.no_grad():
            current_x_train, _ = model.autoencoders[layer_number](current_x_train)
            dataloader = torch.utils.data.DataLoader(
                torch.utils.data.TensorDataset(current_x_train),
                batch_size=batch_size,
                shuffle=True
            )
            dataloader_iter = iter(dataloader)
    
    try:
        # Evaluate the model
        with torch.no_grad():
            
            # Get the encoded representations
            encoded, _ = model(x_train)
            encoded = encoded.to('cpu')
            
            # Evaluate average nmi and ncut over several kmeans runs
            cumulated_nmi = 0
            cumulated_ncut = 0
            for _ in tqdm.tqdm(range(nb_kmeans_tests), desc="computing avg nmi and ncut"):
                kmeans = sklearn.cluster.KMeans(n_clusters=len(set(y)), algorithm="lloyd", random_state=random.randint(0, 10000,), n_init='auto')
                y_pred = kmeans.fit_predict(encoded)
                cumulated_nmi += sklearn.metrics.normalized_mutual_info_score(y, y_pred)
                cumulated_ncut += compute_ncut(s, y_pred)
            avg_nmi = cumulated_nmi / nb_kmeans_tests
            avg_ncut = cumulated_ncut / nb_kmeans_tests
            
            # Print average nmi and ncut
            print("[*] average nmi =", avg_nmi)
            print("[*] average ncut =", avg_ncut)
            
            # If average nmi is better than the best so far, update best_avg_nmi
            if avg_nmi > best_avg_nmi:
                best_avg_nmi = avg_nmi
            
            # If average ncut is better than the best so far, update best_avg_ncut and its corresponding average nmi (i.e. best_avg_ncut_avg_nmi)
            if avg_ncut < best_avg_ncut:
                best_avg_ncut = avg_ncut
                best_avg_ncut_avg_nmi = avg_nmi
    
    except sklearn.exceptions.ConvergenceWarning:
        print("[!] KMeans did not converge (not enough distinct points) --> Returning inf for avg_ncut")
        avg_ncut = float('inf')

    # Return avg_ncut as the objective to minimize
    return avg_ncut


# Set global parameters
nb_epochs_per_layer_pool = [10, 100, 500, 1000, 2500, 5000]
nb_kmeans_tests = 100
nb_trials = 20
device = ('cuda' if torch.cuda.is_available() else 'cpu'); print("[*] using device:", device)
x_train = torch.tensor(nts, dtype=torch.float32).to(device)
batch_size = x_train.shape[0]

# Set globals to track best results
best_avg_nmi = 0.0
best_avg_ncut = float('inf')
best_avg_ncut_avg_nmi = 0.0

# Run optuna study
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(sampler=sampler, direction="minimize")
optuna.logging.set_verbosity(optuna.logging.WARNING)
study.optimize(objective, n_trials=nb_trials)

# Display the best results
print("========================================================")
print("========================================================")
print("[*] best avg nmi =", best_avg_nmi)
print("[*] best avg ncut =", best_avg_ncut)
print("[*] best avg ncut avg nmi =", best_avg_ncut_avg_nmi)

[*] using device: cuda

trial 0----------------------------
> hidden dims = [128, 64]
> rho = 0.0013292918943162175
> beta = 566.9849511478844


computing avg nmi and ncut: 100%|██████████| 100/100 [00:00<00:00, 377.99it/s]


[*] average nmi = 0.260493922075894
[*] average ncut = 1.9609009154117445

trial 1----------------------------
> hidden dims = [128, 64]
> rho = 0.00029380279387035364
> beta = 0.060252157362038566


computing avg nmi and ncut: 100%|██████████| 100/100 [00:00<00:00, 415.94it/s]


[*] average nmi = 0.5118703712076803
[*] average ncut = 1.9158383986290632

trial 2----------------------------
> hidden dims = [128, 64]
> rho = 0.006358358856676255
> beta = 34.70266988650411


...

[*] average nmi = 0.6769578707111265
[*] average ncut = 1.8968657880161661

trial 18----------------------------
> hidden dims = [128, 64]
> rho = 0.03934866506014746
> beta = 20.750670919082573


computing avg nmi and ncut: 100%|██████████| 100/100 [00:00<00:00, 420.08it/s]


[*] average nmi = 0.6516708494718527
[*] average ncut = 1.9084256127998076

trial 19----------------------------
> hidden dims = [128, 64]
> rho = 0.010795471446152866
> beta = 0.11855939250721566


computing avg nmi and ncut: 100%|██████████| 100/100 [00:00<00:00, 341.28it/s]

[*] average nmi = 0.652153713229684
[*] average ncut = 1.8980507870545607
[*] best avg nmi = 0.7081083835306905
[*] best avg ncut = 1.896417754703889
[*] best avg ncut avg nmi = 0.6783239712402153
